# Lab 6: Calibration Standards & Automation of SOLT/ TRL Calibrations

## Content
- Background
   - SOLT
   - TRL
- Design
- Simulation
- Fabrication
- Calibration
    

## Background

A vector network analyzer requires that a measurement calibration be performed before error-corrected measurements can be made. For two-port measurements, the calibration algorithm used will determine the appropriate calibration kit, known either as SOLT or TRL/TRL*.

Traditional full two calibration calibration typically utilizes three impedance and one transmission standards to define the calibrated reference plane. These standards, typically a Short, Open, Load, and Thru, make up the SOLT calibration kit.

Another two port calibration type utilizes a minimum of three standards to define the calibrated reference plane. The measured parameters of the Thru, Reflect, and Line standards in a TRL/TRL* calibration kit provides the same information as a SOLT calibration via a different algorithm.

Depending on the availability of calibration standards and the functionality of the network analyzer, one may use either calibration kit.


## Design

SOLT calibration kits are very difficult to design, so we will only be desiging a TRL kit.

The design of a TRL calibration kit is fairly straightforward. The lengths and widths of the thru, reflect, and line microstrips will need to be calculated.  

We will design a TRL kit for a 62mil FR-4 substrate operating at 2.45GHZ. Calculate the length and width for a matched transmission line between two 50Ω ports.  This can be done by hand or using a microstrip calculator.

#### Thru

- The width calculated should be the same for all three microstrips.

#### Reflect

- The reflect length is half of the thru length. 

#### Line

- Calculated the line length by adding λ/4 to the thru length.

![Pic](TRL.png)


## Simulation

Open ANSYS Electronics Desktop, create a new project titled yourName_TRLKit. 

### 1) Create a circuit design for the thru, reflect and line microstrips. 

*Verify the S-parameters show total reflection for the reflect line and total transmission for the thru. 

  
### 2) Create an HFSS simulation for the three TRL microstrips, again verifying the S-parameter results.  The HFSS circuits should look similar to the images below. Compare the simulated and measured results for the TRL kit.

*Remember to make the substrate much wider than the microstrip, and place the waveports at the end of the strips.

#### Thru
![Pic](thru.png)

#### Line
![Pic](line.png)

#### Reflect
![Pic](reflect.png)





## Calibration

## Deembedding

Deembedding is done in post processing.  This uses algorithms that remove the effect of the fixture on the tests.  This can be done using the scikit rf package in Python.  To begin, save the touchstone files of the S-parameter values from the network analyzer.

### 3) Create a python program that uses scikit RF to perform a TRL and SOLT calibration.

#### SOLT

This is an example of SOLT calibration deembedding using scikit RF.


In [2]:
import skrf as rf
from skrf.calibration import OnePort

## created necessary data for Calibration class

# a list of Network types, holding 'ideal' responses
my_ideals = [\
        rf.Network('ideal/short.s1p'),
        rf.Network('ideal/open.s1p'),
        rf.Network('ideal/load.s1p'),
        ]

# a list of Network types, holding 'measured' responses
my_measured = [\
        rf.Network('measured/short.s1p'),
        rf.Network('measured/open.s1p'),
        rf.Network('measured/load.s1p'),
        ]

## create a Calibration instance
cal = rf.OnePort(\
        ideals = my_ideals,
        measured = my_measured,
        )


## run, and apply calibration to a DUT

# run calibration algorithm
cal.run()

# apply it to a dut
dut = rf.Network('my_dut.s1p')
dut_caled = cal.apply_cal(dut)
dut_caled.name =  dut.name + ' corrected'

# plot results
dut_caled.plot_s_db()
# save results
dut_caled.write_touchstone()

FileNotFoundError: [Errno 2] No such file or directory: 'ideal/short.s1p'

#### TRL

This is an example of TRL calibration deembedding using scikit RF.

In [4]:
import skrf as rf
from skrf.calibration import TRL

%matplotlib inline
from skrf.plotting import stylely
stylely()

T = rf.Network('trl_data/thru.s2p')
R = rf.Network('trl_data/reflect.s2p')
L = rf.Network('trl_data/line.s2p')

switch_terms = (rf.Network('trl_data/forward switch term.s1p'),
                rf.Network('trl_data/reverse switch term.s1p'))

measured = [T,R,L]
trl = TRL(measured = measured,
          switch_terms = switch_terms)

dut_raw = rf.Network('trl_data/mismatched line.s2p')
dut_corrected = trl.apply_cal(dut_raw)
dut_corrected.plot_s_db()


FileNotFoundError: [Errno 2] No such file or directory: 'trl_data/thru.s2p'

## Calibration through the Network Analyzer

Deembedding through python is often unnecessary, since many network analyzers can be calibrated internally before testing.  This means that the measured data is already adjested to account for the fixtures, and doesnt need to be further adjusted in post-processing. 

### 4) Calibrate the network analyzer using the SOLT calibration kit.  Take a picture of the network analyzer's screen before and after calibration. Observe the differences in the graphs.

This is a simple process, on the network analyzer select 'response' --> 'cal' --> 'calibration wizard' --> 'unguided' and then select either 'TRL' ot 'SOLT' depending on which sort of calibration kit you're using. Then follow the directions on the screen, attaching the calibration fixtures until the machine is fully calibrated. 


### SOLT Kit
![Pic](calkit.png)

#### The type of calibration fixture is labeled on the back.  Be sure you attach male to female componenets and vice versa.
![Pic](cal.png)

